<div>
<img src="marg3naFinale.png" width="100" height="100" align="left">
</div>

#### The Python notebook with the Data Cleaning, Model initailization, & Model fine-tuning code

#### Credits:<br>
##### Done By:

1- Yousif Adel Khalil (202000238)<br>
2- Mostafa Samer Dorrah (202000125)<br>
3- Ahmed Ashraf Mohamed (202000563)<br>
4- Abdallah Ashraf Elsaadany (202000467)<br>
5- Mahmoud Mohamed Mahmoud (202002556)<br>
6- Mariam Wael Mohamed (202001259)<br>

##### Under The Supervision of:

Dr. Ghada Khoriba<br>
&<br>
Eng. Salma Gamal


### - Import Needed Libraries

In [2]:
import pandas as pd
import numpy as np
import re
from transformers import BertForQuestionAnswering, BertTokenizer, Trainer, TrainingArguments, GPT2LMHeadModel
from arabert.aragpt2.grover.modeling_gpt2 import GPT2LMHeadModel
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer
from transformers import TextDataset,DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments, AutoModelWithLMHead
import torch
from transformers import pipeline
import evaluate
from transformers import GPT2TokenizerFast, pipeline
from transformers import GPT2LMHeadModel
from arabert.aragpt2.grover.modeling_gpt2 import GPT2LMHeadModel
import lmppl
from transformers import AutoTokenizer, AutoModel, utils

from evaluate import logging
import warnings
warnings.filterwarnings('ignore')

### - Data Cleaning

In [3]:
df = pd.read_csv('C:\\Users\\joead\\OneDrive\\Desktop\\FatwaData\\88kData (1).csv')

In [4]:
df = df.drop('ID', axis=1)

In [5]:
df = df.dropna()

In [6]:
df

,title,ques,ans
0,الشراء عن طريق البنوك الربوية حرام,ما حكم شراء السيارة بالأقساط المريحة من بنك رب...,الحمد لله والصلاة والسلام على رسول الله وعلى آ...
1,لابد من حصر الورثة لتتم قسمة التركة بصورة شرعية,امرأة توفيت ولها مبلغ من المال ، ولها زوج طاعن...,الحمد لله والصلاة والسلام على رسول الله وآله و...
2,لا ينبغي للموظف أن يخالف الشرع ليحصل على إجازة,ما حكم الإجازات المرضية فى حالة مرض أولادي ؟,الحمد لله والصلاة والسلام على رسول الله وعلى آ...
3,أحكام سجود السهو,ما هو سجود السهو ، كيفيته، ووقته؟ وما حكم من ش...,الحمد لله والصلاة والسلام على رسول الله وآله و...
4,إذا أرضعت الأخت أخاها صار ابناً لها,هل يمكن للأخت أن ترضع أخاها، في حال مرض الأم؟ ...,الحمد لله والصلاة والسلام على رسول الله وآله و...
...,...,...,...
88601,الطلاق على الإبراء... طلاق أم خلع,زوجتي أرادت الطلاق وأنا رفضت لأنه لا يوجد مبرر...,الحمد لله والصلاة والسلام على رسول الله وعلى آ...
88602,نتف الحواجب داخل في النمص,أنا أذنبت بأني نتفت شعرة واحدة من حاجبي لأنني ...,الحمد لله والصلاة والسلام على رسول الله وعلى آ...
88603,حكم لبس الباروكة أمام الرجال الأجانب,أنا أم لطفلين أعمارهما 8 سنوات وسنتين، أعيش بب...,الحمد لله والصلاة والسلام على رسول الله وعلى آ...
88604,ارجع إلى شركتك وأعلمها بما هو كائن,أنا أعمل في شركة ما، وعندي شقة فى مدينة من الم...,الحمد لله والصلاة والسلام على رسول الله وعلى آ...


In [7]:
ll = []

In [8]:
ll+=list(df[df['ans'].str.contains('انظر إلى الفتوى')].index)

In [9]:
ll+=list(df[df['ans'].str.contains('رقم:')].index)

In [10]:
ll+=list(df[df['ans'].str.contains('فتوى رقم')].index)

In [11]:
len(ll)

82113

In [12]:
df = df.drop(index=ll, axis=0)

In [13]:
df

,title,ques,ans
0,الشراء عن طريق البنوك الربوية حرام,ما حكم شراء السيارة بالأقساط المريحة من بنك رب...,الحمد لله والصلاة والسلام على رسول الله وعلى آ...
1,لابد من حصر الورثة لتتم قسمة التركة بصورة شرعية,امرأة توفيت ولها مبلغ من المال ، ولها زوج طاعن...,الحمد لله والصلاة والسلام على رسول الله وآله و...
2,لا ينبغي للموظف أن يخالف الشرع ليحصل على إجازة,ما حكم الإجازات المرضية فى حالة مرض أولادي ؟,الحمد لله والصلاة والسلام على رسول الله وعلى آ...
3,أحكام سجود السهو,ما هو سجود السهو ، كيفيته، ووقته؟ وما حكم من ش...,الحمد لله والصلاة والسلام على رسول الله وآله و...
4,إذا أرضعت الأخت أخاها صار ابناً لها,هل يمكن للأخت أن ترضع أخاها، في حال مرض الأم؟ ...,الحمد لله والصلاة والسلام على رسول الله وآله و...
...,...,...,...
88588,هل رفض الأم تبرع ولدها لها بكليته يعد قتلا للنفس,أمي مصابة بقصور الكلى وهي بذلك مجبرة على التصف...,الحمد لله والصلاة والسلام على رسول الله وعلى آ...
88590,من يئس من إيصال الأمانة لصاحبها,سؤالي هو: إذا كان في ذمة الشخص دين (بضاعة) ولم...,الحمد لله والصلاة والسلام على رسول الله وعلى آ...
88596,حكم عدم مراعاة ترتيب السور في قراءة الصلاة,هل يصح للإمام التنكيس في الصلاة بمعنى أن يقرأ ...,الحمد لله والصلاة والسلام على رسول الله وعلى آ...
88597,تضييق الزوج على أهل زوجته من زيارتها ليس من ال...,أنا متزوجة من 12 سنة، وأعاني من أن زوجي إذا زا...,الحمد لله والصلاة والسلام على رسول الله وعلى آ...


In [14]:
df[df['ans'].str.contains('นู')]

,title,ques,ans
84488,ู ุฐุงูุจ ุงูุนูู ุงุก ูู ุงูุฃุญู ุจุง...,ู ุงุชุช ุฃู ู ููุง ุฃุฐูุฑ ุฅู ูุงูุช ...,ุงูุญู ุฏ ููู ูุงูุตูุงุฉ ูุงูุณูุงู...


In [15]:
df = df.drop(index=84488, axis=0)

In [16]:
def extract_arabic_and_english_numerics(text):
    arabic_pattern = r'[^a-zA-Z\s0-9]+'
    numeric_pattern = r'\d+'
    combined_pattern = f'({arabic_pattern}|{numeric_pattern})'
    
    matches = re.findall(combined_pattern, text)
    
    extracted_text = ' '.join(matches)
    
    return extracted_text

In [17]:
#Applying the function on the ['ans'] column
df['ans'] = df['ans'].apply(extract_arabic_and_english_numerics)

In [18]:
#Applying the function on the ['ques'] column
df['ques'] = df['ques'].apply(extract_arabic_and_english_numerics)

In [19]:
#Applying the function on the ['title'] column
df['title'] = df['title'].apply(extract_arabic_and_english_numerics)

In [20]:
listt = []

In [21]:
#Visualize the answers with length less than 118 charachter
for k in range(len(df)):
    
    if len(df['ans'].iloc[k]) < 118:
        print(df['ans'].iloc[k])

, ; . , , - , . . , , , . . ' . . .
الحمد لله والصلاة والسلام على رسول الله وعلى آله وصحبه أما بعد:
الحمد لله والصلاة والسلام على رسول الله وعلى آله وصحبه أما بعد:
الحمد لله والصلاة والسلام على رسول الله وعلى آله وصحبه أما بعد:
الحمد لله والصلاة والسلام على رسول الله وعلى آله وصحبه، أما بعـد: فقد سبق بيان حكمه بالفتوى برقم 7051 .
الحمد لله والصلاة والسلام على رسول الله وعلى آله وصحبه، أما بعـد: 12 - جوابه في الذي قبله.
الحمد لله والصلاة والسلام على رسول الله وعلى آله وصحبه، أما بعـد:
الحمد لله والصلاة والسلام على رسول الله وعلى آله وصحبه، أما بعـد:
الحمد لله والصلاة والسلام على رسول الله وعلى آله وصحبه، أما بعـد:
الحمد لله والصلاة والسلام على رسول الله وعلى آله وصحبه، أما بعـد:
الحمد لله والصلاة والسلام على رسول الله وعلى آله وصحبه، أما بعـد:
الحمد لله والصلاة والسلام على رسول الله وعلى آله وصحبه، أما بعـد:
الحمد لله والصلاة والسلام على رسول الله وعلى آله وصحبه، أما بعـد:
الحمد لله والصلاة والسلام على رسول الله وعلى آله وصحبه، أما بعـد:
الحمد لله والصلاة والسلام على رسول الله وعلى آله 

In [22]:
#Dropping every row that contains an answer with length less than 118 charachter
for k in range(len(df)):
    
    if len(df['ans'].iloc[k]) < 118:
        listt.append(k)


In [23]:
listt = list(df.iloc[listt].index)

In [24]:
df = df.drop(index=listt, axis=0)

In [25]:
#Making all the dataset columns as a string, and dropping duplicates from the dataset.
df = df.astype(str)
df.drop_duplicates(inplace=True)

In [26]:
df = df[0:20000]

In [27]:
df

,title,ques,ans
0,الشراء عن طريق البنوك الربوية حرام,ما حكم شراء السيارة بالأقساط المريحة من بنك رب...,الحمد لله والصلاة والسلام على رسول الله وعلى آ...
1,لابد من حصر الورثة لتتم قسمة التركة بصورة شرعية,امرأة توفيت ولها مبلغ من المال ، ولها زوج طاعن...,الحمد لله والصلاة والسلام على رسول الله وآله و...
2,لا ينبغي للموظف أن يخالف الشرع ليحصل على إجازة,ما حكم الإجازات المرضية فى حالة مرض أولادي ؟,الحمد لله والصلاة والسلام على رسول الله وعلى آ...
3,أحكام سجود السهو,ما هو سجود السهو ، كيفيته، ووقته؟ وما حكم من ش...,الحمد لله والصلاة والسلام على رسول الله وآله و...
4,إذا أرضعت الأخت أخاها صار ابناً لها,هل يمكن للأخت أن ترضع أخاها، في حال مرض الأم؟ ...,الحمد لله والصلاة والسلام على رسول الله وآله و...
...,...,...,...
32774,أقوال العلماء فيمن سبقه الحدث في الصلاة,شيخنا الفاضل هل هناك قول في حالة إذا فسد وضوء ...,الحمد لله والصلاة والسلام على رسول الله وعلى آ...
32777,معنى النق في اللغة,ما معنى النق؟,الحمد لله والصلاة والسلام على رسول الله وعلى آ...
32781,اختبار عقائد الناس,هل يجوز اختبار عقائد الناس بأسئلة معينة كـ (أي...,الحمد لله والصلاة والسلام على رسول الله وعلى آ...
32782,الفرق بين الصديق والحبيب,ما هو الفرق بين الصديق والحبيب، وهل كل صديق مم...,الحمد لله والصلاة والسلام على رسول الله وعلى آ...


In [28]:
#Save the cleaned version of the data to the Machine as .csv file
df.to_csv("Finalized_Data2.csv",index=False)

In [29]:
df = pd.read_csv("Finalized_Data2.csv")

### - Model Intialization & Data Adjustement for Model Fine-Tuning

In [30]:
df["full_text"]=" question: " + df["title"]+df["ques"]+" answer: "+df["ans"]

In [31]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df["full_text"],test_size=0.10, shuffle=True, random_state=42)

In [32]:
train.to_csv(r'train_dataset1.txt', header=None, index=None, sep=' ', mode='a')
test.to_csv(r'test_dataset1.txt', header=None, index=None, sep=' ', mode='a')

In [33]:
model = AutoModelWithLMHead.from_pretrained("aubmindlab/aragpt2-base")
tokenizer = AutoTokenizer.from_pretrained("aubmindlab/aragpt2-base")
tokenizer.add_tokens(['<question>', '<answer>'])
train_path = 'train_dataset1.txt'
test_path = 'test_dataset1.txt'

### - (ARAGPT2) Fine-Tuning

In [34]:
def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)

    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)

    #mlm = Masked Language Modeling (Not our task)
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

train_dataset,test_dataset,data_collator = load_dataset(train_path,test_path,tokenizer)

In [35]:
len(train_dataset)

61800

In [36]:
torch.cuda.empty_cache()

In [37]:
training_args = TrainingArguments(
    output_dir="./islamweb", 
    overwrite_output_dir=True, 
    num_train_epochs=30,
    per_device_train_batch_size=20, 
    per_device_eval_batch_size=10,  
    save_steps=20000, 
    do_eval=True,
    evaluation_strategy='epoch'
    )

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

In [37]:
trainer.train()

***** Running training *****
  Num examples = 61800
  Num Epochs = 30
  Instantaneous batch size per device = 20
  Total train batch size (w. parallel, distributed & accumulation) = 20
  Gradient Accumulation steps = 1
  Total optimization steps = 92700
  Number of trainable parameters = 134994432


Epoch,Training Loss,Validation Loss
1,3.206600,2.824554
2,2.971600,2.716062
3,2.852300,2.662099
4,2.737500,2.623260
5,2.665900,2.600914
6,2.599000,2.580395
7,2.528900,2.569813
8,2.491100,2.560133
9,2.409000,2.549747
10,2.364600,2.548801


***** Running Evaluation *****
  Num examples = 6890
  Batch size = 10
***** Running Evaluation *****
  Num examples = 6890
  Batch size = 10
***** Running Evaluation *****
  Num examples = 6890
  Batch size = 10
***** Running Evaluation *****
  Num examples = 6890
  Batch size = 10
***** Running Evaluation *****
  Num examples = 6890
  Batch size = 10
***** Running Evaluation *****
  Num examples = 6890
  Batch size = 10
Saving model checkpoint to ./islamweb\checkpoint-20000
Configuration saved in ./islamweb\checkpoint-20000\config.json
Model weights saved in ./islamweb\checkpoint-20000\pytorch_model.bin
***** Running Evaluation *****
  Num examples = 6890
  Batch size = 10
***** Running Evaluation *****
  Num examples = 6890
  Batch size = 10
***** Running Evaluation *****
  Num examples = 6890
  Batch size = 10
***** Running Evaluation *****
  Num examples = 6890
  Batch size = 10
***** Running Evaluation *****
  Num examples = 6890
  Batch size = 10
***** Running Evaluation *****
 

TrainOutput(global_step=92700, training_loss=2.2866350057492877, metrics={'train_runtime': 30032.4321, 'train_samples_per_second': 61.733, 'train_steps_per_second': 3.087, 'total_flos': 1.21108856832e+17, 'train_loss': 2.2866350057492877, 'epoch': 30.0})

In [38]:
trainer.save_model()

Saving model checkpoint to ./islamweb
Configuration saved in ./islamweb\config.json
Model weights saved in ./islamweb\pytorch_model.bin


In [36]:
import gradio as gr
from transformers import  pipeline

generation_pipeline = pipeline('text-generation',model="islamweb", tokenizer='aubmindlab/aragpt2-base')


def prediction(str):

    str=generation_pipeline(str,max_length=300)[0]['generated_text']
    start_index = str.index('answer:')
    if 'والله أعلم' in str:
        stop_index = str.index('والله أعلم')
        str = str[start_index+8:stop_index+11].strip()
        
    else:
        str = str[start_index+8:].strip()

    return str

demo = gr.Interface(fn=prediction,outputs=gr.Textbox(label="الاجابه"),inputs=gr.Textbox(label="ما هو سؤالك؟"),title="مَرْجِعْنا",css="body {background-color: green}")

demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://4778f25b1b2188aa15.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


In [38]:
prompts = ['حكم الربا','حكم شرب الخمر','حكم قطع الرحم','حكم الحجاب للمرأة','فضل غض البصر','ما مدا اهمية الصلاة','فضل اذكار الصباح والمساء',
           'كيفية الوضوء','حكم القمار','هل يجوز المسح على الخفين في الوضوء'
           ,'كيفية غسل الجنابة','الجهاد في سبيل الله','فضل ليلة القدر','ماهي سنن الفطرة','الحكمة من تحريم لحم الخنزير']

generated_answers = []

In [ ]:
for i in range(len(prompts)):
    gen_text = prediction(prompts[i])
    if len(gen_text) > 11:
        generated_answers.append(gen_text)
    else:
        gen_text = prediction(prompts[i])
        generated_answers.append(gen_text)
        

In [42]:
scorer = lmppl.LM(model="islamweb")

Using pad_token, but it is not set yet.


In [44]:
scorer.get_perplexity(generated_answers)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.30it/s]


[3.215757822550823,
 1.4879683179125194,
 1.6962214543621599,
 1.8482672411570735,
 nan,
 1.7327981213005563,
 1.7669644965319715,
 2.4445907944204994,
 1.4855601228427697,
 2.3121547601270063,
 2.0925044639021775]

In [45]:
generated_answers[0]

'الحمد لله والصلاة والسلام على رسول الله وعلى آله وصحبه أما بعد: فلا يجوز التعامل مع البنوك الربوية مطلقاً، لما في ذلك من التعاون معها على الإثم والعدوان، والله تعالى يقول: وَلا تَعَاوَنُوا عَلَى الْأِثْمِ والْعُدْوَانِ [المائدة: 2 ]. ويجب عليك التوبة إلى الله تعالى مما فعلت، والندم على ما صدر منك والعزم على عدم العودة إليه مرة أخرى، وعليك أن تتوب إلى الله عز وجل توبة نصوحاً قبل أن يفجأك الأجل المحتوم، قال تعالى: فَالَتَجَزَاهَا اليْنْدَةُ أَ إَهم بَبَارَاثَيلَ رَ اللَكَ كَ قَرَفَاتَ ثَامَضَائَاءَ سَاةَ النَاحم نَاسِيمَ السِحٍ {البقرة: 45 }. ولا يجوز لك الاقتراض من البنك الربوي بأي'

In [39]:
from transformers import AutoTokenizer, AutoModel, utils

model = AutoModelWithLMHead.from_pretrained("Abo3Adel/Marge3na", output_attentions=True)

input_text = "ما حكم الربا وهل هو من الكبائر"

inputs = tokenizer.encode(input_text, return_tensors='pt')
outputs = model(inputs)
attention = outputs[-1]
tokens = tokenizer.decode(inputs[0]).split()


from bertviz import head_view
head_view(attention, tokens)

<IPython.core.display.Javascript object>